# Подгрузка пакетов и данных

In [1]:
cd ..

/Users/n.barsukov/WebstormProjects/ml-bug-detector


In [2]:
import json

from gensim.models.word2vec import Word2Vec

from shared.consts import FUNCTIONS_JSON_PROPERTIES_NAMES, PATH_TO_FUNCTIONS_STORAGE_JSON, PATH_TO_WORD2VEC_MODEL

Загрузим уже построенную модель Word2Vec

In [3]:
word2VecModel = Word2Vec.load(PATH_TO_WORD2VEC_MODEL)

Загрузим JSON, в котором мы сложили названия функций (и названия их арггументов)

In [4]:
functions_storage_dictionary = json.load(open(PATH_TO_FUNCTIONS_STORAGE_JSON))

# Отделение объявления функций от их использования

Функции с одним и тем же названием могут быть объявлены по-разному, поэтому задания функций будут склдываться в словарь по принципу:

Имя репозитория + имя функции

In [25]:
SEPARATOR_REPO_FUNCNAME = '___'

def get_key_in_func_creation_dict(func_name, file_path):
    paths_segments = file_path.split('/')
    
    if (len(paths_segments) > 2):
        return paths_segments[1] + SEPARATOR_REPO_FUNCNAME + func_name
    else:
        return func_name

In [26]:
def find_all_functions_creations(func_storage_dic):
    func_creation_dic = {}
    
    for filePath in func_storage_dic.keys():
        for function in func_storage_dic[filePath]:
            if (
                function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_TYPE] == 'FunctionDeclaration' or
                function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_TYPE] == 'FunctionExpression'
               ):
                func_name = function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME]
                func_creation_dict_key = get_key_in_func_creation_dict(func_name, filePath)
                if func_creation_dict_key in func_creation_dic:
                    pass
                    #print(func_creation_dict_key, 'уже есть в словаре')
                func_creation_dic[func_creation_dict_key] = {
                    f"{FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES}" : function[FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES],
                    f"{FUNCTIONS_JSON_PROPERTIES_NAMES.FILE_PATH}": filePath
                }
    
    return func_creation_dic

In [27]:
func_creation_dic = find_all_functions_creations(functions_storage_dictionary)

А теперь находим все ВЫЗОВЫ функций и смотрим, насколько их аргументы отличны объявления функции

In [29]:
def find_all_func_executions(func_storage_dic, func_creation_dic):
    func_exec_storage = []
    
    for filePath in func_storage_dic.keys():
        for function in func_storage_dic[filePath]:
            func_name = function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME]
            func_creation_dict_key = get_key_in_func_creation_dict(func_name, filePath)
            
            if (
                function[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_TYPE] == 'CallExpression' and # должна быть вызовом функции
                len(function[FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES]) > 0 and # должна иметь хоть один аргумент
                func_creation_dict_key in func_creation_dic # должна быть в нашем словаре
            ):
                function[FUNCTIONS_JSON_PROPERTIES_NAMES.FILE_PATH] = filePath
                func_exec_storage.append(function)
    
    return func_exec_storage

In [30]:
func_exec_storage = find_all_func_executions(functions_storage_dictionary, func_creation_dic)

In [31]:
print('Размер словаря с объявлениями функций: ', len(func_creation_dic))
print('Размер массива с вызовами функций: ', len(func_exec_storage))

Размер словаря с объявлениями функций:  18626
Размер массива с вызовами функций:  2730


Словарь с объявлениями функций - словарь, в котором ключ - название функции, а значение – массив с названиями переданных аргументов.

Пример: функция с названием **confirmDelete**

In [32]:
func_creation_dic[get_key_in_func_creation_dict('confirmDelete', 'scripts/Block8/PHPCI/public/assets/js/phpci.js')]

{'argumentsNames': ['url', 'subject', 'reloadAfter'],
 'filePath': 'scripts/Block8/PHPCI/public/assets/js/phpci.js'}

Массив с вызовами функций состоит из словарей следующего формата

In [33]:
list(filter(lambda func_exec: func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME] == 'confirmDelete',func_exec_storage))

[]

# Алгоритм проверки схожести аргументов

In [35]:
def checkIfWordInWord2VecDict(word, word2VecVocabulary = word2VecModel.wv):
    return word in word2VecVocabulary

def get_args_similarity(exec_func_name, exec_func_args_names, func_path, func_dict, word2VecModel):
    """Определяет схожесть названий аргументов в вызове функции с названиями аргументов в объявлении функции"""
    args_similarity = []
    
    func_in_func_creation_dic_key = get_key_in_func_creation_dict(exec_func_name, func_path)
    
    for argIndex in range(len(exec_func_args_names)):
        
        if argIndex > len(func_dict[func_in_func_creation_dic_key][FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES]) - 1:
            args_similarity.append(None)
            continue
        
        exec_arg = exec_func_args_names[argIndex]
        create_arg = func_dict[get_key_in_func_creation_dict(exec_func_name, func_path)][FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES][argIndex]
        
        if (checkIfWordInWord2VecDict(exec_arg) and checkIfWordInWord2VecDict(create_arg)):
            args_similarity.append(word2VecModel.wv.similarity(exec_arg, create_arg))
        else:
            args_similarity.append(None)
        
    return args_similarity

Посмотрим, как это работает на примере

In [38]:
example_func_exec = list(filter(lambda func: func[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME].endswith('updateLine'), func_exec_storage))
example_func_exec = example_func_exec[0]

func_name = example_func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME]
func_path = example_func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.FILE_PATH]
args_names = example_func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES]

print('Вызов функции:', func_name, 'c аргументами', args_names)
func_in_func_creation_dic_key = get_key_in_func_creation_dict(func_name, example_func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.FILE_PATH])
print('Функция же была объявлена с словаре с аргументами:', func_creation_dic[func_in_func_creation_dic_key][FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES])

get_args_similarity(func_name, args_names, func_path, func_creation_dic, word2VecModel)

Вызов функции: updateLine c аргументами ['lastLine', 'lastSpans', 'estimateHeight']
Функция же была объявлена с словаре с аргументами: ['line', 'text', 'markedSpans', 'estimateHeight']


[0.44814897, -0.022563308, 0.33035174]

# Реализация на всем массиве вызовов функции

Идея максимально проста. Проходим по массиву вызовов функции.

- По названию функции определяем в словаре с объявлениями функций, с какими названиями аргументов была объявлена эта функция
- Для каждой пары аргументов (название аргумента в момент вызовы функции и название аргумента в момент объявление функции) считаем близость их названий в векторной форме с помощью уже построенной модели word2Vec (это будет значение 0 до 1)
- Если эта схожесть меньше заданного порога (threshold), то выбрасываем предупреждение пользователю

### Threshold

In [46]:
MIN_SIMILARITY_THRESHOLD = 0.01

Порог не должен быть большой, потому что модели важно достичь высокого precision, а не recall.

Если наш алгоритм будет часто беспокоить пользователя по пустякам, предупреждая об ошибках там, где их нет (false positive), то нашим алгоритмом просто никто не будет пользоваться. Нужно беспокоить программиста только прям в самых вероятных случаях наличия ошибки. Ведь программисты люди ленивые и занятые... 

In [48]:
for func_exec in func_exec_storage:
    func_name = func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.FUNC_NAME]
    args_names = func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES]
    func_path = func_exec[FUNCTIONS_JSON_PROPERTIES_NAMES.FILE_PATH]
    
    args_similarities = get_args_similarity(func_name, args_names, func_path, func_creation_dic, word2VecModel)
    is_below_threshold_args_similarities = list(map(lambda x: x and x < MIN_SIMILARITY_THRESHOLD, args_similarities))
    
    if (any(is_below_threshold_args_similarities)):
        print('В файле', func_path, 'проверь:')
        print('Была вызвана функция', func_name, 'с аргументами', args_names)
        print('Но вот объявлена функция была со следующими аргументами:')
        print(func_creation_dic[get_key_in_func_creation_dict(func_name, func_path)][FUNCTIONS_JSON_PROPERTIES_NAMES.ARGS_NAMES])
        print('_____________')

В файле scripts/Benvie/Node.js-Ultra-REPL/lib/utility/introspect.js проверь:
Была вызвана функция define с аргументами ['Descriptor']
Но вот объявлена функция была со следующими аргументами:
['o', 'p', 'v']
_____________
В файле scripts/Benvie/continuum/experimental/op-codegen.js проверь:
Была вызвана функция define с аргументами ['BitfieldT']
Но вот объявлена функция была со следующими аргументами:
['o', 'p', 'v']
_____________
В файле scripts/Benvie/continuum/experimental/op-codegen.js проверь:
Была вызвана функция define с аргументами ['PointerT']
Но вот объявлена функция была со следующими аргументами:
['o', 'p', 'v']
_____________
В файле scripts/Benvie/continuum/experimental/op-codegen.js проверь:
Была вызвана функция define с аргументами ['PointerT', 'PointeeType']
Но вот объявлена функция была со следующими аргументами:
['o', 'p', 'v']
_____________
В файле scripts/Benvie/continuum/experimental/op-codegen.js проверь:
Была вызвана функция define с аргументами ['EnumT']
Но вот об

В файле scripts/bpartridge/graphviz.js/graphviz.js проверь:
Была вызвана функция _dup_proto с аргументами ['$g', '$2']
Но вот объявлена функция была со следующими аргументами:
['$g', '$proto']
_____________
В файле scripts/bpartridge/graphviz.js/graphviz.js проверь:
Была вызвана функция _free с аргументами ['$20']
Но вот объявлена функция была со следующими аргументами:
['$mem']
_____________
В файле scripts/bpartridge/graphviz.js/graphviz.js проверь:
Была вызвана функция _aglexinit с аргументами ['$2']
Но вот объявлена функция была со следующими аргументами:
['$fp', '$mygets']
_____________
В файле scripts/bpartridge/graphviz.js/graphviz.js проверь:
Была вызвана функция _copydict с аргументами ['$8', '$5']
Но вот объявлена функция была со следующими аргументами:
['$from', '$to']
_____________
В файле scripts/bpartridge/graphviz.js/graphviz.js проверь:
Была вызвана функция _copydict с аргументами ['$15', '$11']
Но вот объявлена функция была со следующими аргументами:
['$from', '$to']
_

В файле scripts/darlingjs/darlingjs/vendor/box2dEmscripten/box2d-dev.js проверь:
Была вызвана функция __ZN12b2MouseJoint12SetFrequencyEf с аргументами ['$3', '$4']
Но вот объявлена функция была со следующими аргументами:
['$this', '$hz']
_____________
В файле scripts/darlingjs/darlingjs/vendor/box2dEmscripten/box2d-dev.js проверь:
Была вызвана функция __ZN7b2Joint11SetUserDataEPv с аргументами ['$4', '$5']
Но вот объявлена функция была со следующими аргументами:
['$this', '$data']
_____________
В файле scripts/darlingjs/darlingjs/vendor/box2dEmscripten/box2d-dev.js проверь:
Была вызвана функция __ZN12b2MouseJoint15SetDampingRatioEf с аргументами ['$3', '$4']
Но вот объявлена функция была со следующими аргументами:
['$this', '$ratio']
_____________
В файле scripts/darlingjs/darlingjs/vendor/box2dEmscripten/box2d-dev.js проверь:
Была вызвана функция __ZN7b2Joint11SetUserDataEPv с аргументами ['$4', '$5']
Но вот объявлена функция была со следующими аргументами:
['$this', '$data']
________